## Load Dependencies

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import sqlite3
import config
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, func, inspect, Column, Integer, String, Float
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

Base = declarative_base()

## Create sqlite engine, reflect tables

In [3]:
#Add
#create engine to olympics.sqlite
engine = create_engine("sqlite:///olympics.sqlite", echo=False)

## Reflect Database into ORM classes

In [3]:
#Base = automap_base()
Base = declarative_base()
# reflect the tables
#Base.prepare(engine, reflect=True)

## Extract

### Store CSVs into DataFrames

In [2]:
athlete_events = "Resources/athlete_events.csv"
athlete_events_df = pd.read_csv(athlete_events)
athlete_events_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Gold,Silver,Bronze
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,0,0,0
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,0,0,0
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,0,0,0
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,1,0,0
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,0,0,0


## Transform

### Clean athlete events data 

In [3]:
athlete_events_df=athlete_events_df.fillna(0)
athlete_events_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Gold,Silver,Bronze
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,0,0,0
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,0,0,0
2,3,Gunnar Nielsen Aaby,M,24.0,0.0,0.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,0,0,0
3,4,Edgar Lindenau Aabye,M,34.0,0.0,0.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,1,0,0
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,0,0,0


### Check Data Types

In [4]:
athlete_events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 17 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     271116 non-null  float64
 4   Height  271116 non-null  float64
 5   Weight  271116 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Gold    271116 non-null  int64  
 15  Silver  271116 non-null  int64  
 16  Bronze  271116 non-null  int64  
dtypes: float64(3), int64(5), object(9)
memory usage: 35.2+ MB


### Convert float64 to int

In [5]:
athlete_events_df[['Age','Height','Weight']] = athlete_events_df[['Age','Height','Weight']].astype(int)

### Select DF Columns to Import

In [6]:
athlete_events_df2 = athlete_events_df[["Name","Sex","Age","Height","Weight","Team","NOC","Games","Year","Season",\
                                        "City","Sport","Event","Gold","Silver","Bronze"]]
athlete_events_df2.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Gold,Silver,Bronze
0,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,0,0,0
1,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,0,0,0
2,Gunnar Nielsen Aaby,M,24,0,0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,0,0,0
3,Edgar Lindenau Aabye,M,34,0,0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,1,0,0
4,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,0,0,0


In [7]:
#Rename NOC to Country 
athlete_events_df2 = athlete_events_df2.rename(
    columns={"NOC":"Country"})

In [8]:
athlete_events_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 16 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Name     271116 non-null  object
 1   Sex      271116 non-null  object
 2   Age      271116 non-null  int32 
 3   Height   271116 non-null  int32 
 4   Weight   271116 non-null  int32 
 5   Team     271116 non-null  object
 6   Country  271116 non-null  object
 7   Games    271116 non-null  object
 8   Year     271116 non-null  int64 
 9   Season   271116 non-null  object
 10  City     271116 non-null  object
 11  Sport    271116 non-null  object
 12  Event    271116 non-null  object
 13  Gold     271116 non-null  int64 
 14  Silver   271116 non-null  int64 
 15  Bronze   271116 non-null  int64 
dtypes: int32(3), int64(4), object(9)
memory usage: 30.0+ MB


## Load

### Create class for sql database

In [14]:
class Athlete_Events(Base):
    __tablename__ = "Athlete_Events"
    ID = Column(Integer, primary_key=True)
    Name = Column(String(255))
    Sex = Column(String(255))
    Age = Column(Integer)
    Height = Column(Integer)
    Weight = Column(Integer)
    Team = Column(String(255))
    Country = Column(String(255))
    Games = Column(String(255))
    Year = Column(Integer)
    Season = Column(String(255))
    City = Column(String(255))
    Sport = Column(String(255))
    Event = Column(String(255))
    Gold = Column(Integer)
    Silver = Column(Integer)
    Bronze = Column(Integer)

### Connect to local database

In [15]:
#Create engine connection
#engine = create_engine("sqlite:///olympics.sqlite", echo=False)
engine = create_engine("sqlite:///olympics.sqlite")
Base.metadata.create_all(engine)
session = Session(engine)
conn = engine.connect()

### Upload Athlete_Events df to SQLite database

In [38]:
athlete_events_df2.to_sql("Athlete_Events", engine, if_exists="append", index=False)

### Test Connection

In [17]:
con = sqlite3.connect("olympics.sqlite")
query ="SELECT COUNTRY,SPORT, MEDAL from Athlete_Events"
country_medals_df = pd.read_sql_query(query, con)
country_medals_df.head()

,Country,Sport,Medal
0,CHN,Basketball,0
1,CHN,Judo,0
2,DEN,Football,0
3,DEN,Tug-Of-War,Gold
4,NED,Speed Skating,0


## Analysis for visualization

### Postgres Database

In [39]:
host = "localhost"
user = "postgres"
password = "postgres"
port = 5432

url = 'postgresql+psycopg2://'+user+":"+ "postgres" + "@" + host + ":" + "5432" + "/"

database_name = "Olympics"

engine = create_engine(url)
conn = engine.connect()
conn.execute("commit")
conn.execute ("DROP DATABASE IF EXISTS " + database_name)
conn.execute("commit")
conn.execute("CREATE DATABASE " + database_name)
conn.close()
engine.dispose()

In [34]:
#Athlete_Events = Athlete_Events.reset_index()
#Athlete_Events = Athlete_Events.rename(columns = {"index": "ID"})
#athlete_events_df2 = athlete_events_df2.drop(columns = "ID")

In [40]:
#Use "" Around Athlete_Events when querying from Postgres
database_url = url + database_name
Olympics_engine = create_engine(database_url)
athlete_events_df2.to_sql('Athlete_Events', Olympics_engine, if_exists='replace', index=False)

#conn = Olympics_engine.connect()
#conn.execute('alter Athlete_Events add id serial primary key')
#conn.execute("commit")
#conn.close()
#olympics_engine.dispose()

In [4]:
#Create engine connection
#engine = create_engine("sqlite:///olympics.sqlite", echo=False)
engine = create_engine("sqlite:///olympics.sqlite")
Base.metadata.create_all(engine)
session = Session(engine)
conn = engine.connect()